# 0. Импорты


In [ ]:
!pip install nltk

In [ ]:
!pip install tensorflow Cython matplotlib

In [ ]:
!pip install git+https://github.com/lopuhin/python-adagram.git

In [ ]:
!pip install pymystem3 pymorphy2

In [5]:
! pip install corus

In [196]:
! pip install simple-elmo

In [37]:
import adagram
import nltk
nltk.download('stopwords')
from simple_elmo import ElmoModel
import pandas as pd
from lxml import html
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return ' '.join(words)

def tokenize(text):
    return token.tokenize(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100.nonorm.slim.joblib


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  67.0M      0  0:00:20  0:00:20 --:--:-- 48.2M


# 1. Получение значений

Значения брала из словаря МАС

In [9]:
words = ['лист', 'номер', 'звезда', 'среда', 'клетка']

In [27]:
meanings = pd.read_csv('meanings.csv', sep='\t')

In [38]:
meanings['lemmatized'] = meanings['meaning'].apply(normalize)

In [61]:
meanings = meanings.rename(columns={'meaning' : 'sentence', 'lemmatized' : 'lemmatized_sentence'})

In [62]:
meanings.head()

,word,sentence,lemmatized_sentence
0,лист,Тонкая зеленая пластинка различной формы на че...,тонкий зелёный пластинка различный форма на че...
1,лист,"Тонкий, плотный кусок или пласт какого-л. мате...",тонкий плотный кусок или пласт какой л материал
2,лист,"Единица измерения, применяемая в издательском ...",единица измерение применять в издательский и п...
3,лист,"Документ, удостоверяющий что-л. или содержащий...",документ удостоверять что л или содержать како...
4,номер,орядковое число предмета в ряду других однород...,орядковый число предмет в ряд другой однородный


# 2. Отбор предложений из корпусов

Я взяла все корпуса, в которых уже есть морфологическая разметка, так как лемматизация вручную занимала очень много времени.

In [ ]:
! wget https://github.com/UniversalDependencies/UD_Russian-GSD/raw/master/ru_gsd-ud-dev.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-GSD/raw/master/ru_gsd-ud-test.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-dev.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-test.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-Taiga/raw/master/ru_taiga-ud-dev.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-Taiga/raw/master/ru_taiga-ud-test.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-Taiga/raw/master/ru_taiga-ud-train.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-PUD/raw/master/ru_pud-ud-test.conllu

In [11]:
from corus import load_ud_gsd, load_ud_taiga, load_ud_syntag, load_ud_pud

In [12]:
contexts = {word: [] for word in words }

In [13]:
ud_corpora = [load_ud_gsd('ru_gsd-ud-dev.conllu'), load_ud_gsd('ru_gsd-ud-test.conllu'), load_ud_syntag('ru_syntagrus-ud-dev.conllu'), load_ud_syntag('ru_syntagrus-ud-test.conllu'), load_ud_taiga('ru_taiga-ud-dev.conllu'), load_ud_taiga('ru_taiga-ud-test.conllu'), load_ud_pud('ru_pud-ud-test.conllu')]

In [14]:
def get_contexts_ud(corpus):
  for sentence in corpus:
    lemmas = [token.lemma for token in sentence.tokens]
    lemmas = [x for x in lemmas if x != None]
    sentence_lemmatized = ' '.join(lemmas)
    for word in words:
      if word in lemmas:
        contexts[word].append((sentence.text, sentence_lemmatized))


In [15]:
for c in ud_corpora:
  get_contexts_ud(c)

In [ ]:
! wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/GIKRYA_texts_new.zip
!unzip GIKRYA_texts_new.zip
!rm GIKRYA_texts_new.zip
! wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar
! unrar x RNC_texts.rar
! rm RNC_texts.rar
! wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/OpenCorpora_Texts.rar
! unrar x OpenCorpora_Texts.rar
! rm OpenCorpora_Texts.rar


In [17]:
from corus import load_morphoru_gicrya, load_morphoru_rnc, load_morphoru_corpora

In [18]:
morph_eval_corpora = [load_morphoru_gicrya('gikrya_new_test.out'), load_morphoru_rnc('RNCgoldInUD_Morpho.conll'), load_morphoru_corpora('unamb_sent_14_6.conllu')]

In [19]:
def get_contexts_mrphev(corpus):
  for sentence in corpus:
    lemmas = [token.lemma for token in sentence.tokens]
    for word in words:
      if word in lemmas:
        contexts[word].append((' '.join(token.text for token in sentence.tokens), ' '.join(lemmas) ))

In [20]:
for c in morph_eval_corpora:
  try:
    get_contexts_mrphev(c)
  except:
    print(c)

<generator object parse_morphoru_rnc at 0x7e3c98705b60>


In [21]:
word_sentence_lemma_pairs = [
    (word, sentence, lemmatized_sentence)
    for word, sentences in contexts.items()
    for sentence, lemmatized_sentence in sentences
]
df = pd.DataFrame(word_sentence_lemma_pairs, columns=['word', 'sentence', 'lemmatized_sentence'])

In [22]:
df.to_csv('contexts.csv')

# 3. Выделение значений

In [23]:
df.head()

,word,sentence,lemmatized_sentence
0,лист,"Нижние листья -- в розетке, немного кожистые, ...","нижний лист -- в розетка , немного кожистый , ..."
1,лист,"Но дождь уже кончился, и Сергей вышел в сад ка...","но дождь уже кончиться , и Сергей выйти в сад ..."
2,лист,Отсутствие многолетнего опыта не позволяло кон...,отсутствие многолетний опыт не позволять конст...
3,лист,"Кусты рябины в садике, желтые листья по осени;...","куст рябина в садик , желтый лист по осень ; л..."
4,лист,"Солила огурцы с укропом, чесноком, тмином, дуб...","солить огурец с укроп , чеснок , тмин , дубовы..."


In [24]:
for w, s in contexts.items():
    print(w, len(s))

лист 158
номер 185
звезда 119
среда 154
клетка 61


## 3.1 AdaGram

In [40]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [42]:
def disambiguate(row):
  means = vm.disambiguate(row.word, row.sentence)
  n = np.argmax(means)
  return n

In [43]:
df['ada_sense'] = df.apply(disambiguate, axis=1)

In [44]:
df

,word,sentence,lemmatized_sentence,ada_sense
0,лист,"Нижние листья -- в розетке, немного кожистые, ...","нижний лист -- в розетка , немного кожистый , ...",5
1,лист,"Но дождь уже кончился, и Сергей вышел в сад ка...","но дождь уже кончиться , и Сергей выйти в сад ...",2
2,лист,Отсутствие многолетнего опыта не позволяло кон...,отсутствие многолетний опыт не позволять конст...,5
3,лист,"Кусты рябины в садике, желтые листья по осени;...","куст рябина в садик , желтый лист по осень ; л...",2
4,лист,"Солила огурцы с укропом, чесноком, тмином, дуб...","солить огурец с укроп , чеснок , тмин , дубовы...",5
...,...,...,...,...
672,клетка,Посадила героя чеченской войны Буданова в клет...,посадить герой чеченский война Буданов в клетк...,2
673,клетка,В патогенезе гепатита С иммунопатологический к...,в патогенез гепатит С иммунопатологический ком...,2
674,клетка,На этапе острой инспекции репродукция вируса п...,на этап острый инспекция репродукция вирус про...,4
675,клетка,Интерфероны активируют иммунокомпетентные клет...,интерферон активировать иммунокомпетентный кле...,2


## 3.2 Elmo


In [ ]:
!wget http://vectors.nlpl.eu/repository/20/196.zip

In [46]:
from simple_elmo import ElmoModel

In [47]:
model = ElmoModel()

In [48]:
model.load('196.zip')

'The model is now loaded.'

In [49]:
def get_elmo_vectors(word, contexts, model):
    tokens = [c.split() for c in contexts]
    all_vectors = model.get_elmo_vectors(tokens)
    word_vecs = []
    for i in range(len(contexts)):
        try:
            word_vecs.append(all_vectors[i][tokens[i].index(word)])
        except ValueError:
            continue
    return word_vecs

In [50]:
result_list = []

In [51]:
for word in df['word'].unique():
    elmo_vectors = get_elmo_vectors(word, list(df[df['word'] == word]['lemmatized_sentence']), model)

    elmo_df = pd.DataFrame({
        'word': word,
        'lemmatized_sentence': list(df[df['word'] == word]['lemmatized_sentence']),
        'elmo_vectors': elmo_vectors
    })
    result_list.append(elmo_df)

elmo_df = pd.concat(result_list, ignore_index=True)

In [52]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [64]:
k_models = dict()

In [67]:
for word in elmo_df['word'].unique():
    elmo_vectors = elmo_df[elmo_df['word'] == word]['elmo_vectors'].tolist()
    if len(elmo_vectors) > 0:
        X = [v.tolist() for v in elmo_vectors]
        k_range = range(2, 10)
        silhouette_scores = []
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42)
            clusters = kmeans.fit_predict(X)
            silhouette_scores.append(silhouette_score(X, clusters))
        optimal_k = k_range[silhouette_scores.index(max(silhouette_scores))]
        kmeans = KMeans(n_clusters=optimal_k, random_state=42)
        k_models[word] = kmeans
        clusters = kmeans.fit_predict(X)
        clusters = clusters.astype(int) + 1
        elmo_df.loc[elmo_df['word'] == word, 'cluster'] = clusters


In [57]:
elmo_df

,word,lemmatized_sentence,elmo_vectors,cluster
0,лист,"нижний лист -- в розетка , немного кожистый , ...","[0.11416055262088776, -0.39366525411605835, -0...",1.0
1,лист,"но дождь уже кончиться , и Сергей выйти в сад ...","[-0.6904487609863281, -0.22607259452342987, -0...",2.0
2,лист,отсутствие многолетний опыт не позволять конст...,"[0.3626002073287964, -0.2874602675437927, -0.5...",2.0
3,лист,"куст рябина в садик , желтый лист по осень ; л...","[-0.34352195262908936, -0.036001384258270264, ...",2.0
4,лист,"солить огурец с укроп , чеснок , тмин , дубовы...","[0.36924123764038086, -0.1444670557975769, -0....",2.0
...,...,...,...,...
672,клетка,посадить герой чеченский война Буданов в клетк...,"[-0.21635252237319946, -0.430106520652771, 0.4...",1.0
673,клетка,в патогенез гепатит С иммунопатологический ком...,"[0.17164772748947144, -0.16519957780838013, 0....",2.0
674,клетка,на этап острый инспекция репродукция вирус про...,"[0.0975864976644516, -0.2555580735206604, 0.59...",2.0
675,клетка,интерферон активировать иммунокомпетентный кле...,"[0.16248098015785217, -0.5286366939544678, 0.6...",2.0


In [58]:
df = df.merge(elmo_df[['cluster']], left_index=True, right_index=True, how='left')

In [59]:
df.to_csv('wsd.csv')

# 4. Кластеризация словарных значений

#  

In [ ]:
meanings['ada_sense'] = meanings.apply(disambiguate, axis=1)

Так как для контекстуальных эмбеддингов нужно вхождение слова в предложение, добавлю в начало каждого словарного определения "X -- это"

In [69]:
def change_definition(definition, word):
  return f"{word} это {definition}"

In [72]:
meanings['elmo_lemmatized'] = meanings.apply(lambda row: change_definition(row['lemmatized_sentence'], row['word']), axis=1)

In [113]:
meanings['elmo_vector'] = None
meanings['elmo_cluster'] = None

In [138]:
for index, row in tqdm(meanings.iterrows(), total=20):
  contexts = [row.elmo_lemmatized] + list(elmo_df[elmo_df.word == row.word]['lemmatized_sentence'])
  elmo_vector = get_elmo_vectors(row.word, contexts, model)[0]
  meanings.at[index, 'elmo_vector'] = elmo_vector
  cluster = k_models[row.word].predict([elmo_vector])
  meanings.at[index, 'elmo_cluster'] = int(cluster) + 1

  0%|          | 0/20 [00:00<?, ?it/s]

Уже на этом этапе видно, что elmo отнес разные значения в разные кластеры только для слова 'среда' (и то несколько неожиданным образом), AdaGram же некоторые толкования отнесла к разным значениям:

In [194]:
meanings.groupby(['word', 'ada_sense']).agg({'sentence' : 'count'})

sentence
word   ada_sense          
звезда 0                 3
клетка 2                 3
       4                 1
лист   2                 1
       5                 3
номер  4                 6
среда  1                 4

In [195]:
meanings.groupby(['word', 'elmo_cluster']).agg({'sentence' : 'count'})

sentence
word   elmo_cluster          
звезда 3                    3
клетка 2                    4
лист   1                    4
номер  1                    6
среда  2                    2
       3                    2

In [139]:
meanings.to_csv('auto_definitions.csv')

# 5. Разметка

При отборе примеров буду опираться на кластеры, приписанные adagram

In [ ]:
chosen_meanings = {
    'лист' : [5, 2],
    'номер' : [4],
    'звезда' : [0],
    'среда' : [1],
    'клетка' : [2, 4]
}

In [147]:
def choose_random_rows(group):
    return group.sample(n=5)
result_df = df.groupby(['word', 'ada_sense'], group_keys=False).apply(choose_random_rows)


In [149]:
result_df.to_csv('to_label.csv')

# 6. Оценка accuracy

При разметке я ставила 1, если лейбл примера совпадает с лейблом толкования, и 0 в ином случае

In [152]:
res = pd.read_csv('manually_labeled.csv')

Качество двух подходов невысокое и отличается незначительно. Думаю, причина может быть как в недостатке словарных толкований (слишком короткие и абстрактные), так и в качестве самих предложений, поскольку они поступают на вход модели без более широкого контекста, который часто необходим, чтобы разграничить разные употребления.

In [158]:
res.agg({'ada_label' : 'mean', 'elmo_label' : 'mean'})

ada_label     0.575
elmo_label    0.400
dtype: float64

Оба метода выделяют меньше значений, чем указано в словарной статье, но это вполне ожидаемо, поскольку словарные толкования иногда бывают слишком специфичными; даже в том случае, если модель выделяет несколько кластеров, значения попадают не во все из них:

In [182]:
df.groupby(['word', 'ada_sense']).agg({'ada_sense' : 'count'})

ada_sense
word   ada_sense           
звезда 0                100
       2                 19
клетка 2                 41
       4                 20
лист   2                 66
       5                 92
номер  4                185
среда  1                154

In [183]:
df.groupby(['word', 'cluster']).agg({'cluster' : 'count'})

cluster
word   cluster         
звезда 1.0            6
       2.0           81
       3.0           32
клетка 1.0           18
       2.0           43
лист   1.0           43
       2.0          115
номер  1.0           67
       2.0          118
среда  1.0           15
       2.0           93
       3.0           46

Интересно, что AdaGram выделила всего одно значение для слова *среда*, хотя это омоним, у которого, кажется, точно есть минимум 2 значения (день недели и окружение). Elmo решила, что 'день недели' и 'Социально-бытовая обстановка, в которой протекает жизнь человека; окружение' -- это одно значение